## Convert HEIC to JPG

In [3]:
import os
from PIL import Image
import pillow_heif

# --- CONFIG ---
INPUT_FOLDER = r"D:/Pill_Identification/dataset/Source/HEIC"
OUTPUT_FOLDER = r"D:/Pill_Identification/dataset/Source/JPG"

# Iterate over all .HEIC files
for filename in os.listdir(INPUT_FOLDER):
    if filename.lower().endswith(".heic"):
        heic_path = os.path.join(INPUT_FOLDER, filename)
        jpg_filename = os.path.splitext(filename)[0] + ".jpg"
        jpg_path = os.path.join(OUTPUT_FOLDER, jpg_filename)
        if os.path.exists(jpg_path) is False:
            print(f"📂 Converting {filename} -> {jpg_filename}")

            # Read HEIC and convert
            heif_file = pillow_heif.read_heif(heic_path)
            image = Image.frombytes(
                heif_file.mode,
                heif_file.size,
                heif_file.data,
                "raw",
            )

            image.save(jpg_path)
        else:
            print(f"file is already exist. Skipping")
            continue

print("✅ Conversion complete!")


📂 Converting IMG_E4379.HEIC -> IMG_E4379.jpg
📂 Converting IMG_E4380.HEIC -> IMG_E4380.jpg
📂 Converting IMG_E4381.HEIC -> IMG_E4381.jpg
📂 Converting IMG_E4382.HEIC -> IMG_E4382.jpg
📂 Converting IMG_E4383.HEIC -> IMG_E4383.jpg
📂 Converting IMG_E4384.HEIC -> IMG_E4384.jpg
📂 Converting IMG_E4385.HEIC -> IMG_E4385.jpg
📂 Converting IMG_E4386.HEIC -> IMG_E4386.jpg
📂 Converting IMG_E4387.HEIC -> IMG_E4387.jpg
📂 Converting IMG_E4388.HEIC -> IMG_E4388.jpg
📂 Converting IMG_E4389.HEIC -> IMG_E4389.jpg
📂 Converting IMG_E4390.HEIC -> IMG_E4390.jpg
📂 Converting IMG_E4391.HEIC -> IMG_E4391.jpg
📂 Converting IMG_E4392.HEIC -> IMG_E4392.jpg
📂 Converting IMG_E4393.HEIC -> IMG_E4393.jpg
📂 Converting IMG_E4394.HEIC -> IMG_E4394.jpg
📂 Converting IMG_E4395.HEIC -> IMG_E4395.jpg
📂 Converting IMG_E4396.HEIC -> IMG_E4396.jpg
📂 Converting IMG_E4397.HEIC -> IMG_E4397.jpg
📂 Converting IMG_E4398.HEIC -> IMG_E4398.jpg
📂 Converting IMG_E4399.HEIC -> IMG_E4399.jpg
📂 Converting IMG_E4400.HEIC -> IMG_E4400.jpg
📂 Converti

# Split into folders

In [4]:
import os
import shutil

# --- CONFIGURATION ---
input_folder = r"D:/Pill_Identification/dataset/Source/JPG"
output_folder = r"D:/Pill_Identification/dataset/Pill_jpg_2025_Original"

os.makedirs(output_folder, exist_ok=True)

# Get all jpg files and sort them to keep order consistent
files = [f for f in os.listdir(input_folder) if f.lower().endswith('.jpg')]
files.sort()

current_id = 72
id_image_count = 0  # count images within this id (to know when to reset)
group_count = 0     # used to switch MT/MS every 3 images

# separate counters for MT and MS
mt_counter = 0
ms_counter = 0

for idx, filename in enumerate(files):
    # Determine MT/MS every 3 images
    group_type = "MT" if (group_count // 3) % 2 == 0 else "MS"

    if group_type == "MT":
        mt_counter += 1
        number_in_group = mt_counter
    else:
        ms_counter += 1
        number_in_group = ms_counter

    new_filename = f"{current_id}_{group_type}_{number_in_group:02d}.jpg"

    # Create folder structure <id>/<group_type>/Mau
    id_folder = os.path.join(output_folder, str(current_id))
    group_folder = os.path.join(id_folder, group_type, "Mau")
    os.makedirs(group_folder, exist_ok=True)

    # Copy (or move) file
    src = os.path.join(input_folder, filename)
    dst = os.path.join(group_folder, new_filename)
    shutil.copy2(src, dst)  # <-- use shutil.move if you want to move instead

    # Update counters
    id_image_count += 1
    group_count += 1

    if current_id == 15:
        if id_image_count == 12:
            current_id += 1
            id_image_count = 0
            group_count = 0
            mt_counter = 0
            ms_counter = 0

    # Reset after 18 images (new id -> restart all counters)
    if id_image_count == 18:
        current_id += 1
        id_image_count = 0
        group_count = 0
        mt_counter = 0
        ms_counter = 0

print("✅ Done! MT/MS numbers are cumulative per group type.")

✅ Done! MT/MS numbers are cumulative per group type.


In [12]:
from PIL import Image, ImageOps

# Paths to files
mask_path = "D:/Pill_Identification/background_removal_DL/test_data/images/u2net_results/IMG_E3042.png"  # mask (white = keep, black = remove)
image_path = "D:/Pill_Identification/background_removal_DL/test_data/images/input/IMG_E3042.jpg"  # pill image
output_path = "pill_masked.png"

# Open images
img = Image.open(image_path).convert("RGB")
mask = Image.open(mask_path).convert("L")  # convert to grayscale

# Apply mask (white = keep, black = transparent)
result = Image.composite(img, Image.new("RGB", img.size, (0, 0, 0)), mask)

# Save result
result.save(output_path)
print(f"✅ Saved masked image to {output_path}")
result.show()

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Pill_Identification/background_removal_DL/test_data/images/input/IMG_E3042.jpg'

In [ ]:
import os

def cascade_rename(folder, old_name, new_name):
    """
    Rename old_name -> new_name in folder.
    If new_name exists, shift that file forward by +1, recursively,
    until all names are unique.
    """
    old_path = os.path.join(folder, old_name)
    if not os.path.exists(old_path):
        print(f"File {old_name} not found.")
        return

    def shift_file(current_name):
        """Shift file to next available numeric name."""
        base, ext = os.path.splitext(current_name)

        # Separate prefix + number
        prefix = ''.join([c for c in base if not c.isdigit()])
        number = ''.join([c for c in base if c.isdigit()])

        if number == "":
            raise ValueError(f"Cannot shift {current_name}: no number in filename")

        next_number = str(int(number) + 1)
        next_name = prefix + next_number + ext
        current_path = os.path.join(folder, current_name)
        next_path = os.path.join(folder, next_name)

        # If next exists, shift that one first (recursive cascade)
        if os.path.exists(next_path):
            shift_file(next_name)

        os.rename(current_path, next_path)
        print(f"Shifted: {current_name} -> {next_name}")

    # If new_name already exists, shift it forward
    if os.path.exists(os.path.join(folder, new_name)):
        shift_file(new_name)

    # Finally, rename the original
    os.rename(old_path, os.path.join(folder, new_name))
    print(f"Renamed: {old_name} -> {new_name}")


# Example usage
folder = r"D:/Pill_Identification/Datasets/JPG"
cascade_rename(folder, "IMG_E3675.jpg", "IMG_E3231.jpg")


Shifted: IMG_E3234.jpg -> IMG_E3235.jpg
Shifted: IMG_E3233.jpg -> IMG_E3234.jpg
Shifted: IMG_E3232.jpg -> IMG_E3233.jpg
Shifted: IMG_E3231.jpg -> IMG_E3232.jpg
Renamed: IMG_E3675.jpg -> IMG_E3231.jpg


In [2]:
import os
from PIL import Image

def resize_images(input_folder, output_folder, size=(256, 256)):
    """
    Recursively resize all images in input_folder to given size
    and save them into output_folder (all in one folder, no subfolders).
    Keeps original filenames, avoids overwriting by adding suffix only if needed.
    """
    os.makedirs(output_folder, exist_ok=True)

    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
                img_path = os.path.join(root, filename)
                try:
                    with Image.open(img_path) as img:
                        # Resize with high-quality resampling
                        img_resized = img.resize(size, Image.LANCZOS)

                        # Build save path
                        base, ext = os.path.splitext(filename)
                        save_path = os.path.join(output_folder, filename)

                        # If file exists, add suffix (_1, _2, ...)
                        counter = 1
                        while os.path.exists(save_path):
                            save_path = os.path.join(output_folder, f"{base}_{counter}{ext}")
                            counter += 1

                        # Save image
                        img_resized.save(save_path)
                        print(f"Resized: {img_path} -> {save_path}")
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

# Example usage:
input_folder = r"D:/Pill_Identification/dataset/Pill_jpg_2025_Original"
output_folder = r"D:/Pill_Identification/dataset/For_model/Pill_jpg_2025_Resized"
resize_images(input_folder, output_folder, size=(640, 640))


Resized: D:/Pill_Identification/dataset/Pill_jpg_2025_Original\0\MS\Mau\0_MS_01.jpg -> D:/Pill_Identification/dataset/For_model/Pill_jpg_2025_Resized\0_MS_01.jpg
Resized: D:/Pill_Identification/dataset/Pill_jpg_2025_Original\0\MS\Mau\0_MS_02.jpg -> D:/Pill_Identification/dataset/For_model/Pill_jpg_2025_Resized\0_MS_02.jpg
Resized: D:/Pill_Identification/dataset/Pill_jpg_2025_Original\0\MS\Mau\0_MS_03.jpg -> D:/Pill_Identification/dataset/For_model/Pill_jpg_2025_Resized\0_MS_03.jpg
Resized: D:/Pill_Identification/dataset/Pill_jpg_2025_Original\0\MS\Mau\0_MS_04.jpg -> D:/Pill_Identification/dataset/For_model/Pill_jpg_2025_Resized\0_MS_04.jpg
Resized: D:/Pill_Identification/dataset/Pill_jpg_2025_Original\0\MS\Mau\0_MS_05.jpg -> D:/Pill_Identification/dataset/For_model/Pill_jpg_2025_Resized\0_MS_05.jpg
Resized: D:/Pill_Identification/dataset/Pill_jpg_2025_Original\0\MS\Mau\0_MS_06.jpg -> D:/Pill_Identification/dataset/For_model/Pill_jpg_2025_Resized\0_MS_06.jpg
Resized: D:/Pill_Identificat

In [ ]:
import os
import shutil

# --- CONFIGURATION ---
input_folder = r"D:/Pill_Identification/Datasets/Resized"
output_folder = r"D:/Pill_Identification/Datasets/Pill_jpg_2025"

# Create output folder (fresh)
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder, exist_ok=True)

# Collect all jpg files from input_folder + subfolders
files = []
for root, _, filenames in os.walk(input_folder):
    for f in filenames:
        if f.lower().endswith(".jpg"):
            files.append(os.path.join(root, f))

# Sort by full path for consistent order
files.sort()

current_id = 0
id_image_count = 0  # count images within this id (to know when to reset)
group_count = 0     # used to switch MT/MS every 3 images

# separate counters for MT and MS
mt_counter = 0
ms_counter = 0

for idx, filepath in enumerate(files):
    filename = os.path.basename(filepath)

    # Determine MT/MS every 3 images
    group_type = "MT" if (group_count // 3) % 2 == 0 else "MS"

    if group_type == "MT":
        mt_counter += 1
        number_in_group = mt_counter
    else:
        ms_counter += 1
        number_in_group = ms_counter

    # New filename (everything goes into one flat folder)
    new_filename = f"{current_id}_{group_type}_{number_in_group:02d}.jpg"
    dst = os.path.join(output_folder, new_filename)

    # Copy (or move) file
    shutil.copy2(filepath, dst)  # <-- use shutil.move if you want to move instead

    # Update counters
    id_image_count += 1
    group_count += 1

    # Reset after 18 images (new id -> restart all counters)
    if id_image_count == 18:
        current_id += 1
        id_image_count = 0
        group_count = 0
        mt_counter = 0
        ms_counter = 0

print("✅ Done! Read all subfolders, renamed, and saved into a single folder.")


✅ Done! Read all subfolders, renamed, and saved into a single folder.
